# Predicting the sale price of bulldozers using machine learning

In this notebook we're going to go through an example machine learning project with the goal og predicting the sale price of 


## 1. Problem definition
How well can we predict the future price of a bulldozer given its characteristics and previous examples of how much similar bulldozers have been sold for?
## 2. Data
https://www.kaggle.com/competitions/bluebook-for-bulldozers/data

The data is downloaded from the Keggle Bluebook for Bulldozers competition:

View and download the benchmark code from Github

For this competition, you are predicting the sale price of bulldozers sold at auctions.

The data for this competition is split into three parts:

Train.csv is the training set, which contains data through the end of 2011.
Valid.csv is the validation set, which contains data from January 1, 2012 - April 30, 2012 You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.
Test.csv is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. Your score on the test set determines your final rank for the competition.
The key fields are in train.csv are:

SalesID: the uniue identifier of the sale
MachineID: the unique identifier of a machine.  A machine can be sold multiple times
saleprice: what the machine sold for at auction (only provided in train.csv)
saledate: the date of the sale
There are several fields towards the end of the file on the different options a machine can have.  The descriptions all start with "machine configuration" in the data dictionary.  Some product types do not have a particular option, so all the records for that option variable will be null for that product type.  Also, some sources do not provide good option and/or hours data.
The machine_appendix.csv file contains the correct year manufactured for a given machine along with the make, model, and product class details. There is one machine id for every machine in all the competition datasets (training, evaluation, etc.).
## 3. Evaluation
The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.

For more on evaluation on project search:
https://www.kaggle.com/competitions/bluebook-for-bulldozers/overview

Note: the goal for most regression evaluation metrics is to minimize the error. For example the goal of this project will be to build a machine learning model which minimizes RMSLE.
## 4. Features
Keggle provides a data dictionary providing all of the features of a data set. You can view this data dictionary on Google Sheets:


In [250]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [251]:
# Import data


In [252]:
df = pd.read_csv("Data/bluebook-for-bulldozers/bluebook-for-bulldozers/TrainAndValid.csv")


C:\Users\USER\AppData\Local\Temp\ipykernel_9296\656879906.py:1: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data/bluebook-for-bulldozers/bluebook-for-bulldozers/TrainAndValid.csv")


MemoryError: Unable to allocate 142. MiB for an array with shape (45, 412698) and data type object

In [ ]:
df

In [ ]:
df.info

In [ ]:
## Visualize
fig, ax= plt.subplots()
ax.scatter(df["saledate"][:1000], df["SalePrice"][:1000]);

In [ ]:
df.SalePrice.plot.hist();

### Parsing Dates

When we work with time series data we want to enrich the time and date component as much as possible. How. We can do that by telling pandas which of our columns has date in it using `parse_dates`



In [ ]:
## Import data again but this time parse dates

df=pd.read_csv("Data/bluebook-for-bulldozers/bluebook-for-bulldozers/TrainAndValid.csv",
              parse_dates=["saledate"] )

In [ ]:
df.saledate.dtype


In [ ]:

df.saledate[:1000]

In [ ]:

df

## Sort DataFrame by saledate

When working with time series data it is best to sort by date

In [ ]:
df.saledate.head(20)

In [ ]:
## Make copy of the original copy
df_tmp=df.copy()
df_tmp

In [ ]:
df_tmp.sort_values(by=["saledate"], inplace=True, ascending=True)
df.saledate.head(20)

In [ ]:
## Make copy of the original copy
df_tmp=df.copy()
df_tmp

### Add daytime parameters for `saledate` column

In [ ]:
df_tmp["saleYear"]=df.saledate.dt.year
df_tmp["saleMonth"]=df.saledate.dt.month
df_tmp["saleDay"]=df.saledate.dt.day
df_tmp["saleDayOfWeek"]= df.saledate.dt.dayofweek
df_tmp["saleDayOfYear"]=df.saledate.dt.dayofyear

In [ ]:
df_tmp.drop("saledate", axis=1, inplace=True)

# 5.Modelling

We've done enough edaa. Now lets do model driven EDA

In [ ]:
## Let's build a machine learning model
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor(n_jobs=-1,
                           random_state=42)

model.fit(df_tmp.drop("SalePrice", axis=1), df_tmp["SalePrice"])

In [ ]:
pd.api.types.is_string_dtype(df_tmp["UsageBand"])

In [ ]:
## Turining data to numbers

In [ ]:
for label,content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
       print(label)

In [ ]:
## This will turn all the string values to categories
for label,content in df_tmp.items():
    if pd.api.types.is_string_dtype(content):
        df_tmp[label] = content.astype("category").cat.as_ordered()


In [ ]:

df_tmp.info()

In [ ]:
## Check missing data

In [ ]:
df_tmp.isnull().sum()/len(df_tmp)

### Save preprocessed data

In [ ]:

## Export current tmp dataframe
df_tmp.to_csv("Data/bluebook-for-bulldozers/bluebook-for-bulldozers/train_tmp.csv",
              index=False )

In [ ]:
## Import preprocessed data
df_tmp= pd.read_csv("Data/bluebook-for-bulldozers/bluebook-for-bulldozers/train_tmp.csv",
              low_memory=False)

df_tmp.head().T

## Fill missing numerical values

In [ ]:



for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
## Check for which numeric columns have null values
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if content.isnull().sum():
            print(label)

In [ ]:
## Fill numeric rows with median
for label,content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum:
            df_tmp[label +" is_missing"] = pd.isnull(content)
            df_tmp[label] = content.fillna(content.median())

In [ ]:
## Check if there are missing numeric values
for label, content in df_tmp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)
        

## Filling and turing categorical variables into numbers

In [ ]:
## Check for coulumns which aren't numerical

for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
## Turn categorical variables into numbers and fill missing values
for label, content in df_tmp.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_tmp[label +"_is_missing"]= pd.isnull(content)
        ## Turn categories to numbers and add +1
        df_tmp[label]= pd.Categorical(content).codes + 1

In [ ]:
df_tmp.info()

In [ ]:
df_tmp.head().T

In [ ]:
df_tmp.isna().sum()

In [ ]:
model= RandomForestRegressor(n_jobs=-1,
                            random_state=42)

##Fit the model
model.fit(df_tmp.drop("SalePrice", axis=1), df_tmp["SalePrice"])

In [ ]:
print(df_tmp.shape)


In [ ]:
model.score(df_tmp.drop("SalePrice", axis=1), df_tmp["SalePrice"])

**Question:** Why doesn't the metric above hold water?

In [ ]:
## Split and validate the data
df_val= df_tmp[df_tmp.saleYear==2012]
df_train= df_tmp[df_tmp.saleYear!=2012]

In [ ]:
X_train, y_train= df_train.drop("SalePrice", axis=1), df_train.SalePrice
X_valid, y_valid= df_val.drop("SalePrice", axis=1), df_val.SalePrice

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

## Building an evaluation function

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_log_error
import numpy as np

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)

    scores = {
        "Training MAE": mean_absolute_error(y_train, train_preds),
        "Validation MAE": mean_absolute_error(y_valid, val_preds),
        "Training RMSLE": rmsle(y_train, train_preds),
        "Validation RMSLE": rmsle(y_valid, val_preds),
        "Training R^2": r2_score(y_train, train_preds),
        "Validation R^2": r2_score(y_valid, val_preds)
    }

    return scores


## Testing your model on a subset(To tune hyperparameters)

In [ ]:
len(X_train)

In [ ]:
## Change max samples value

model=RandomForestRegressor(n_jobs=-1,
                           random_state=42,
                           max_samples=10000)



In [ ]:
%%time

model.fit(X_train, y_train)

In [ ]:
show_scores(model)

## Hyperparameter tuning with RandomizedSearchCV 

In [ ]:
%%time 

from sklearn.model_selection import RandomizedSearchCV

## Different RandomForest hyperparameters

rf_grid= {"n_estimators" : np.arange(10, 100, 10),
          "max_depth" : [None, 3, 5, 10],
          "min_samples_split": np.arange(2, 20,2),
          "min_samples_leaf" : np.arange(1, 20, 2),
          "max_features" : [0.5, 1, "sqrt","auto"],
          "max_samples": [10000]
    
}


##Instanciate a RandomizedSearchCV model
rs_model= RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,
                                                  random_state=42),
                            param_distributions=rf_grid,
                            n_iter=2,
                            cv=5,
                            verbose=True)


rs_model.fit(X_train, y_train)

In [ ]:
rs_model.best_params_


In [ ]:
## Evaluate the Randomized Search Model

show_scores(rs_model)

## Train models with best hyperparameters

***Note***: These were found after 100 iterations of `RandomizedSearchCV`

In [ ]:
ideal_model= RandomForestRegressor(n_estimators=40,
                                  min_samples_leaf=1,
                                  min_samples_split=14,
                                  max_features= 0.5,
                                  n_jobs=-1,
                                  max_samples=None,
                                  random_state=42)
ideal_model.fit(X_train, y_train)

In [ ]:
show_scores(ideal_model)

## Make Predictions on the test data

In [256]:
## Import the test data
df_test= pd.read_csv("Data/bluebook-for-bulldozers/bluebook-for-bulldozers/Test.csv",
                    low_memory=False,
                    parse_dates=["saledate"])
df_test.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1227829,1006309,3168,121,3,1999,3688.0,Low,2012-05-03,580G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1227844,1022817,7271,121,3,1000,28555.0,High,2012-05-10,936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1227847,1031560,22805,121,3,2004,6038.0,Medium,2012-05-10,EC210BLC,...,None or Unspecified,"9' 6""",Manual,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
3,1227848,56204,1269,121,3,2006,8940.0,High,2012-05-10,330CL,...,None or Unspecified,None or Unspecified,Manual,Yes,Triple,NaN,NaN,NaN,NaN,NaN
4,1227863,1053887,22312,121,3,2005,2286.0,Low,2012-05-10,650K,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN


In [258]:
## Make predictions on the test dataset

test_preds= ideal_model.predict(df_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- saledate
Feature names seen at fit time, yet now missing:
- Backhoe_Mounting_is_missing
- Blade_Extension_is_missing
- Blade_Type_is_missing
- Blade_Width_is_missing
- Coupler_System_is_missing
- ...
